In [28]:
import sys
sys.path.append("../../..")

from heritageconnector.datastore import es, index

import utils

In [29]:
data_list = utils.load_jsonld_dump("../data/test_data_None.jsonl")
len(data_list)

297425

In [30]:
utils.get_entities_from_jsonl_doc(data_list[0])

{'uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co8027145',
 'description': 'Poster, British Railways (Southern) poster, (Weekend Engineering Work May Affect Your Journey), 7 & 8 November 1970 and list of station pairs in Kent and Sussex, Turquoise, October 1970, (Other IDs AD3644/B12/191070).',
 'entities': [('British Railways', 'ORG'),
  ('May', 'ORG'),
  ('Engineering', 'ORG'),
  ('Turquoise', 'LOC'),
  ('Kent', 'LOC'),
  ('Sussex', 'LOC'),
  ('AD3644', 'OBJECT')]}

In [50]:
def get_candidates(doc: dict):
    candidate_list = []
    
    # 1. transform to simplified form
    simplified_doc = utils.get_entities_from_jsonl_doc(doc)
    
    # for each mention:
    for ent_mention, ent_type in simplified_doc.entities:
        # 2. look up mention in ES using highlight query
        candidates = look_up_mention(ent_mention)
        
        for candidate in candidates:
            pass
    
        # 4. store mention alongside candidates (with comparison?)
        
        pass
        
def look_up_mention(mention: str, ent_types=["PERSON", "ORG", "FAC", "LOC", "OBJECT"], limit=1000):
    
    query = {
        "query": {
            "multi_match": {
                "query": mention,
                "fields": [f"graph.@hc:entity{t}.@value" for t in ent_types]
            }
        },
        "highlight": {
            "fields": {f"graph.@hc:entity{t}.@value": {} for t in ent_types}
        }
    }

    search_res = es.search(index=index, body=query, size=limit)["hits"]["hits"]

    candidate_info = []
    
    if limit:
        search_res = search_res[0:limit] if limit > len(search_res) else search_res
    
    for item in search_res:
        item_entities = []
        
        for entity_type_key, entity_mention_list in item["highlight"].items():
            item_entities += [(val.replace("<em>", "").replace("</em>", ""), entity_type_key[16:-7]) for val in entity_mention_list]
        
        candidate_info.append({"uri": item['_id'], "description": item["_source"]["data"]["http://www.w3.org/2001/XMLSchema#description"], "entities": item_entities})
    
    return candidate_info

ent_search_res = look_up_mention("victoria")

print(len(ent_search_res))

[i for i in ent_search_res if len(i["entities"])>1]
    
    


760


[{'uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co8001366',
  'description': "Turtle shell, with 'Victoria Hotel' and the arms of Sheffield painted on the back, ex Royal Victoria Hotel, Sheffield.",
  'entities': [('Victoria Hotel', 'ORG'), ('Royal Victoria Hotel', 'ORG')]},
 {'uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co232583',
  'description': 'Poster, British Railways (Southern Region), The Brighton Belle by A N Wolstenholme, 1950.  Coloured lithograph train departure notice, with an images of the Brighton Belle electric multiple unit, giving details of train times.  Weekdays depart London (Victoria)  11am arrive Brighton 12 noon, dep London Victoria 3pm arr Brighton 4pm, dep London Victoria 7pm arr Brighton 8pm, depart Brighton 1.25pm arrive London (Victoria) 2.25pm, dep Brighton 5.25pm arr London Victoria 6.25pm, dep Brighton 8.25pm arr London Victoria 9.25pm, Sundays dep London (Victoria) 11 am arr Brighton 12 noon, dep Brighton 5.25pm arr London

In [52]:
from heritageconnector.best_spacy_pipeline import load_model
from spacy import displacy

In [53]:
nlp = load_model("en_core_web_trf")

spacy tried to use GPU but failed
2021-02-02 15:31:46,175 - hc_nlp.pipeline - INFO - Loading thesaurus from ../../../heritageconnector/../GITIGNORE_DATA/labels_all_unambiguous_types_people_orgs.jsonl
2021-02-02 15:31:47,563 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 1s


In [56]:
displacy.render(nlp("Poster, British Railways (Southern Region), The Brighton Belle by A N Wolstenholme, 1950.  Coloured lithograph train departure notice, with an images of the Brighton Belle electric multiple unit, giving details of train times.  Weekdays depart London (Victoria)  11am arrive Brighton 12 noon, dep London Victoria 3pm arr Brighton 4pm, dep London Victoria 7pm arr Brighton 8pm, depart Brighton 1.25pm arrive London (Victoria) 2.25pm, dep Brighton 5.25pm arr London Victoria 6.25pm, dep Brighton 8.25pm arr London Victoria 9.25pm, Sundays dep London (Victoria) 11 am arr Brighton 12 noon, dep Brighton 5.25pm arr London Victoria 6.25pm, dep London Victoria 7pm arr Brighton 8pm, dep Brighton 8.25pn arr London Victoria 9.25pm. print of the Brighton Belle by A N Wolstinholme. printed at staffords Netherfield. Format: double royal, 40 x 25 inches, 1016 x 635mm."), style="ent")